# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain.agents import tool

In [3]:
@tool # converts the function into a langchain tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

search.name, search.description, search.args

('search',
 'Search for weather online',
 {'query': {'title': 'Query', 'type': 'string'}})

In [4]:
# Define a structure for the input schema
# Important since the LLM use it as a prompt
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

search.name, search.description, search.args

('search',
 'Search for weather online',
 {'query': {'description': 'Thing to search for',
   'title': 'Query',
   'type': 'string'}})

In [5]:
search.run("CDG")

'42f'

## Application 1: Get temperature given a location latitude and longitude

In [6]:
import requests
from datetime import datetime

# Input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(description="Latitude of the location to fetch weather data")
    longitude: float = Field(description="Longitude of the location to fetch weather data")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.now()
    time_list = [datetime.fromisoformat(time_str.replace("Z", "+00:00")) for time_str in results["hourly"]["time"]]
    temperature_list = results["hourly"]["temperature_2m"]
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f"{current_temperature}°C"

In [7]:
get_current_temperature.name, get_current_temperature.description, get_current_temperature.args

('get_current_temperature',
 'Fetch current temperature for given coordinates.',
 {'latitude': {'description': 'Latitude of the location to fetch weather data',
   'title': 'Latitude',
   'type': 'number'},
  'longitude': {'description': 'Longitude of the location to fetch weather data',
   'title': 'Longitude',
   'type': 'number'}})

In [8]:
# Convert the tool to an actual OpenAI function
from langchain_core.utils.function_calling import convert_to_openai_function
get_current_temperature_openai = convert_to_openai_function(get_current_temperature)

In [9]:
from langchain_openai import ChatOpenAI
import json

# Create the request and ask the model to identify the function inputs
latitude = 3.87
longitude = 11.52

ask_weather = """What's the current temperature in:
latitude = {latitude}
longitude = {longitude}
"""

messages = [
    {"role": "user", "content": ask_weather.format(latitude=latitude, longitude=longitude)},
]

model = ChatOpenAI().bind(functions=[get_current_temperature_openai])
response = model.invoke(messages)
messages.append(response)

# Retrieve the inputs and execute the function
args = json.loads(response.additional_kwargs["function_call"]["arguments"])
temperature = get_current_temperature.run(args)

# Add the function output to the conversation and display the final response
messages.append({
    "role": "function",
    "name": response.additional_kwargs["function_call"]["name"],
    "content": temperature,
})

response = model.invoke(messages)
response.content

'The current temperature at latitude 3.87 and longitude 11.52 is 27.7°C.'

## Application 2: Summarise Wikipedia pages

In [10]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia Search and get page summaries"""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary.strip()}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DIsambiguationError,
        ):
            pass
    
    if not summaries:
        return "No good Wikipedia Search Result was found"

    return "\n\n".join(summaries)

In [11]:
search_wikipedia.name, search_wikipedia.description, search_wikipedia.args
search_wikipedia_openai = convert_to_openai_function(search_wikipedia)

In [12]:
result = search_wikipedia.invoke({"query": "langchain"})
print(result)

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.

Page: Sentence em

## Create OpenAI function from an OpenAPI Specification (OAS) 

The OAS defines a standard, language-agnostic interface to HTTP APIs which allows both humans and computers to discover and understand the capabilities of the service without access to source code.

In [13]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [14]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [15]:
spec = OpenAPISpec.from_text(text)
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [16]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)
model.batch([
    "What are three pets names?",
    "Tell me about the pet with id 21",
])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 124, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d432c504-5ac5-43f2-a291-55aaa6da133a-0', usage_metadata={'input_tokens': 124, 'output_tokens': 16, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"21"}}', 'name': 'showPetById'}, 'refusal': None}, response_metadata={'token_usage': {'completion_to

# Routing to choose which function to invoke if applicable

In [18]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    ("user", "{input}"),
])

functions = [get_current_temperature_openai, search_wikipedia_openai]
model = ChatOpenAI(temperature=0).bind(functions=functions)

chain = prompt | model

chain.batch([
    "What is the weather in Yaounde right now?",
    "What is Langchain?",
])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":3.848,"longitude":11.5021}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 111, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-12f0f3a6-f7cb-4c7a-aba4-c70c99a226a7-0', usage_metadata={'input_tokens': 111, 'output_tokens': 24, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'to

In [19]:
# Is it a normal response? A function call? What are the inputs in the 2nd case?
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [20]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()
inputs = [
    {"input": "What is the weather in Yaounde right now?"},
    {"input": "What is Langchain?"},
    {"input": "Hi!"},
]
results = chain.batch(inputs)
results

[AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 3.848, 'longitude': 11.5021}, log="\nInvoking: `get_current_temperature` with `{'latitude': 3.848, 'longitude': 11.5021}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":3.848,"longitude":11.5021}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 111, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-aa6d4f46-4516-4743-be73-3dfaa2e225a6-0', usage_metadata={'input_tokens': 111, 'output_tokens': 24, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

In [21]:
# Route the result properly
from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values["output"]
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].invoke(result.tool_input)
    
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route
chain.batch(inputs)

['27.8°C',
 "Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.\